<div class="alert alert-block alert-success">
    <b><h2>Kategorik Değişken (Categorical Variable)</h2></b> 
    <ul> Kategorik değişkenler, iki veya daha fazla kategoriye sahip olunduğu durumdur.
        <li>3 farklı yaklaşıma değineceğiz
            <ul>
    <br><li> Bu sütunları düşürme, silme
        <li> Label Encoding ile her benzersiz kelimeyi farklı bir sayı ile temsil etme
        <li> One Hot Encoding ile her kategoriyi sütun haline getirerek kodlama
            </ul>
    </ul>
        <img src="https://womaneng.com/wp-content/uploads/2018/09/onehotencoding.jpg"  width=400/>
</div>

#### Bu bölümde işlenecek konular

1. [RandomForestRegression modeli fonksiyon tanımlama](#bir)
2. [Train test diye veri setini ayırma](#iki)
3. [Boş değer içeren sütunların düşürülmesi](#uc)
4. [Kategorik ve numerik sütunların tespit edilmesi](#dort)
5. [Kategorik değerlerin düşürülmesi ve model test edilmesi](#bes)
6. [Label Encoding ve model test edilmesi](#altı)
7. [One Hot Encoding ve model test edilmesi](#yedi)

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
data=pd.read_csv('./Data/melbourne-housing-snapshot/melb_data.csv')

In [8]:
data.head().notnull

<bound method DataFrame.notnull of        Suburb           Address  Rooms Type      Price Method SellerG  \
0  Abbotsford      85 Turner St      2    h  1480000.0      S  Biggin   
1  Abbotsford   25 Bloomburg St      2    h  1035000.0      S  Biggin   
2  Abbotsford      5 Charles St      3    h  1465000.0     SP  Biggin   
3  Abbotsford  40 Federation La      3    h   850000.0     PI  Biggin   
4  Abbotsford       55a Park St      4    h  1600000.0     VB  Nelson   

        Date  Distance  Postcode  ...  Bathroom  Car  Landsize  BuildingArea  \
0  3/12/2016       2.5    3067.0  ...       1.0  1.0     202.0           NaN   
1  4/02/2016       2.5    3067.0  ...       1.0  0.0     156.0          79.0   
2  4/03/2017       2.5    3067.0  ...       2.0  0.0     134.0         150.0   
3  4/03/2017       2.5    3067.0  ...       2.0  1.0      94.0           NaN   
4  4/06/2016       2.5    3067.0  ...       1.0  2.0     120.0         142.0   

   YearBuilt  CouncilArea Lattitude  Longtitu

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

In [5]:
y = data.Price

In [6]:
X=data.drop(columns="Price")

<div id="bir"  style="color:blue;"> 1. RandomForestRegressor modeli oluşturma</div>
<ul>
      <li>Model için n_estimators ile ağaç sayısı belirlenir</li>
    <li>random_state=0 her çalıştırmada verinin her sefer aynı şekilde ayrılması sağlanır <i> from sklearn.ensemble import RandomForestRegressor </i></li>
    <li>Mutlak hata oranını ölçüyoruz ve geri dönderiyoruz
</ul>

In [7]:
def score_dataset(X_train,X_test,y_train,y_test):
    model=RandomForestRegressor(n_estimators=10,random_state=0)
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    return mean_absolute_error(y_test,pred)


<div id="iki"  style="color:blue;"> 2. Train test diye veri setini ayırma</div>
<ul>
    <li>Veriyi doğrulama yapmak için parçalışıyoruz. Verinin %80 train, %20 test için ayrılıyor
    <li> random_state=0 ile her seferinde verinin aynı parçalanması sağlanır
    <li>shuffle=True seçilmezse veriyi en baştan %80 ve geri kalanını %20 test diye ayırır. Shuffle ile veri setinin karıştırılığ parçalanmasını sağlar
</ul>

In [8]:
def train_test(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=2,shuffle=True)
    return X_train, X_test, y_train, y_test

<div id="uc"  style="color:blue;"> 3. Boş değerlerin veri setinden çıkarılması</div>
<ul>
      <li>NaN içeren değerler drop edilir</li>
    <li> axis=1 sütun, axis=0 satır demek. Burda sütunları düşürüyoruz</li>
</ul>

In [9]:
for item in X.columns:
    if X[item].isnull().any():
        print(item)

Car
BuildingArea
YearBuilt
CouncilArea


In [10]:
nan_value_column=[item for item in X.columns if X[item].isnull().any()]

In [11]:
X_=X.drop(nan_value_column,axis=1)

<div id="dort"  style="color:blue;"> 4. Kategorik ve numerik sütunların tespit edilmesi</div>

In [12]:
kategorik_sutunlar=[item for item in X_.columns if X_[item].dtype=="object" and X_[item].nunique()<10]
numerik_sutunlar=[item for item in X_.columns if X_[item].dtype in ["int64","float64"]]
sutunlar=numerik_sutunlar+kategorik_sutunlar
sutunlar

['Rooms',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Landsize',
 'Lattitude',
 'Longtitude',
 'Propertycount',
 'Type',
 'Method',
 'Regionname']

#### Numerik ve kategorik sutunlardan yeni bir dataframe oluşturma

In [13]:
X_all=X_[sutunlar].copy()

In [14]:
X_all.head(3)

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount,Type,Method,Regionname
0,2,2.5,3067.0,2.0,1.0,202.0,-37.7996,144.9984,4019.0,h,S,Northern Metropolitan
1,2,2.5,3067.0,2.0,1.0,156.0,-37.8079,144.9934,4019.0,h,S,Northern Metropolitan
2,3,2.5,3067.0,3.0,2.0,134.0,-37.8093,144.9944,4019.0,h,SP,Northern Metropolitan


<div id="bes"  style="color:blue;"> 5. Kategorik sütunların düşürülmesi ve model testi</div>

In [15]:
X_drop_categorical=X_all.select_dtypes(exclude=["object"])

In [16]:
X_drop_categorical.head(3)

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
0,2,2.5,3067.0,2.0,1.0,202.0,-37.7996,144.9984,4019.0
1,2,2.5,3067.0,2.0,1.0,156.0,-37.8079,144.9934,4019.0
2,3,2.5,3067.0,3.0,2.0,134.0,-37.8093,144.9944,4019.0


In [17]:
X_train,X_test,y_train,y_test=train_test_split(X_drop_categorical,y)

In [18]:
print("Kategorik veriler drop edildiğinde mutlak hata" ,score_dataset(X_train,X_test,y_train,y_test))

Kategorik veriler drop edildiğinde mutlak hata 186872.835537789


<div id="altı"  style="color:blue;"> 6. Label Encoding ve model testi</div>
<ul> 
    <li> Kütüphaneyi ilk başta import ediyoruz <i> from sklearn.preprocessing import LabelEncoder</i>
    <li> Label Encoding ile her tekil değeri bir sayı ile temsil edeceğiz.
</ul>

In [19]:
X_label_enc=X_all.copy()
X_label_enc.head(3)

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount,Type,Method,Regionname
0,2,2.5,3067.0,2.0,1.0,202.0,-37.7996,144.9984,4019.0,h,S,Northern Metropolitan
1,2,2.5,3067.0,2.0,1.0,156.0,-37.8079,144.9934,4019.0,h,S,Northern Metropolitan
2,3,2.5,3067.0,3.0,2.0,134.0,-37.8093,144.9944,4019.0,h,SP,Northern Metropolitan


In [20]:
le=LabelEncoder()

In [21]:
for item in kategorik_sutunlar:
    X_label_enc[item]=le.fit_transform(X_label_enc[item])

#### Son 3 sütun gözüktüğü gibi artık numerik olarak encode edilmiş oldu

In [22]:
X_label_enc.head(3)

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount,Type,Method,Regionname
0,2,2.5,3067.0,2.0,1.0,202.0,-37.7996,144.9984,4019.0,0,1,2
1,2,2.5,3067.0,2.0,1.0,156.0,-37.8079,144.9934,4019.0,0,1,2
2,3,2.5,3067.0,3.0,2.0,134.0,-37.8093,144.9944,4019.0,0,3,2


In [23]:
X_train,X_test,y_train,y_test=train_test_split(X_drop_categorical,y)
print("Kategorik veriler Label Encoding edildiğinde mutlak hata" ,score_dataset(X_train,X_test,y_train,y_test))

Kategorik veriler Label Encoding edildiğinde mutlak hata 194165.72835198825


<div id="yedi"  style="color:blue;"> 7. One Hot Encoding ve model testi</div>
<ul> 
    <li> Kütüphaneyi ilk başta import ediyoruz <i> from sklearn.preprocessing import OneHotEncoder</i>
    <li> One hot encoding her unique değeri sütunlaştırır ve değerle sayısallaştırır
</ul>

In [24]:
X_one_hot=X_all.copy()
X_one_hot.head(2)

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount,Type,Method,Regionname
0,2,2.5,3067.0,2.0,1.0,202.0,-37.7996,144.9984,4019.0,h,S,Northern Metropolitan
1,2,2.5,3067.0,2.0,1.0,156.0,-37.8079,144.9934,4019.0,h,S,Northern Metropolitan


In [25]:
oh_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_kategorik = pd.DataFrame(oh_encoder.fit_transform(X_one_hot[kategorik_sutunlar]))

In [26]:
OH_numeric=num_X_train = X_one_hot.drop(kategorik_sutunlar, axis=1)
OH_kategorik.index=X_one_hot[kategorik_sutunlar].index

In [27]:
OH_X_= pd.concat([OH_numeric, OH_kategorik], axis=1)

In [28]:
X_train,X_test,y_train,y_test=train_test_split(OH_X_,y)
print("Kategorik veriler One Hot Encoding edildiğinde mutlak hata" ,score_dataset(X_train,X_test,y_train,y_test))

Kategorik veriler One Hot Encoding edildiğinde mutlak hata 174465.5889936181
